    # **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

from debugpy.common.log import newline

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [50]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'
coefficient_root = 'coefficient.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [51]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [52]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    training_data = []
    validation_data = []

    # TODO: i don't know probably done
    idx = int(len(data) * split_ratio)
    validation_data = data[idx:]
    training_data = data[:idx]
    
    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [53]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    preprocessedData = []

    # TODO
    # turn gender data to numerical data
    data = np.where(data == 'F', 0, np.where(data == 'M', 10, data))
    data = data.astype(float)
    
    # detect outliers
    Q1 = np.percentile(data, 25, axis=0)
    Q3 = np.percentile(data, 75, axis=0)
    IQR = Q3 - Q1
    
    preprocessedData = np.where((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)), np.nan, data)
    
    # detect missing data and change it to mean value
    '''col_mean = np.nanmean(preprocessedData, axis=0)
    nan_indice = np.isnan(preprocessedData);
    preprocessedData[nan_indice] = np.take(col_mean, np.where(nan_indice)[1])'''
    # detect nan data and remove it
    preprocessedData = preprocessedData[~np.isnan(preprocessedData).any(axis=1)]
    
    return preprocessedData


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [54]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :1]
    y = dataset[:, 1]

    # TODO: Decide on the degree of the polynomial
    degree = 3  # For example, quadratic regression

    # Add polynomial features to X
    # X.size() * 3
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    # 1 * 3
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    try:
        w = np.loadtxt(coefficient_root, delimiter=',')
        return w
    except:
        w = np.zeros(num_dimensions)
    
    # TODO: Set hyperparameters
    num_iteration = 1000000
    learning_rate = 0.0000000000001
    
    print(w)
    
    # Gradient Descent
    m = len(y)  # Number of data points
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_hat = np.dot(X_poly, w) # 1 * X.size()
        error = y_hat - y
        # TODO: Compute gradient
        gradient = (1 / m) * np.dot(X_poly.T, error) # 1 * 3
        # TODO: Update the weights
        w = w - learning_rate * gradient
        
        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 200000 == 0:
            cost = (1 / (2 * m)) * np.sum(np.square(error))
            print(f"Iteration {iteration}, Cost: {cost}")
            print(w)
    
    # store w vector
    with open(coefficient_root, 'w', newline='') as file:
        writer = csv.writer(file)
        for coeff in w:
            writer.writerow([coeff])

    return w


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [55]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO
    degree = 3  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((test_dataset.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, test_dataset ** d))  # Add x^d terms to feature matrix
    
    prediction = np.dot(X_poly, w)

    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to include the coefficients of your model in the report**



In [56]:
def cal_MAPE(y_true, y_pred):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between the true and predicted values.

    Parameters:
    - y_true (numpy.ndarray): The true values.
    - y_pred (numpy.ndarray): The predicted values.

    Returns:
    - mape (float): The MAPE value.
    """
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print(f"Mape value: {mape:.2f}%")
    
    return mape

In [57]:
# TODO
# (1) Split data
trainData, validationData = SplitData(training_datalist, 0.8)

# (2) Preprocess data
preprocessData = PreprocessData(trainData)
validationData = PreprocessData(validationData)

# (3) Train regression model
coeff = Regression(preprocessData)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
validation = MakePrediction(coeff, validationData[:, :1])
mape_val = cal_MAPE(validationData[:, 1], validation)

# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(coeff, testing_datalist)

Mape value: 24.29%


C:\Users\User\AppData\Local\Temp\ipykernel_1912\1992744318.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  data = np.where(data == 'F', 0, np.where(data == 'M', 10, data))


### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [58]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [59]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced_OLS.csv' # Output file will be named as 'lab1_advanced.csv'
coefficient_adv_root = 'coefficient_adv_OLS.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements
degree = 1

used_var = [0, 1, 2, 3, 4, 5, 6]

In [60]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

## Implement the Regression Model

In [61]:
def Regression_adv(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x1 + w[2] * x2 + ...
    """
    np.random.seed(42)

    X = dataset[:, used_var]
    y = dataset[:, -1]

    # TODO: Decide on the degree of the polynomial
    # Add polynomial features to X
    # X.size() * 3
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add terms to feature matrix

    # Initialize coefficients (weights) to zero
    # 1 * 3
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    try:
        w = np.loadtxt(coefficient_adv_root, delimiter=',')
    except:
        w = np.random.randn(num_dimensions)
    
    # TODO: Set hyperparameters
    num_iteration = 500
    learning_rate = 0.01
    batch_size = 16  # Mini-batch size
    decay_rate = 0.9  # RMSprop decay rate (beta)
    epsilon = 1e-8  # Small constant to prevent division by zero
    
    print(w)
    
    # OLS
    w = np.linalg.inv(X_poly.T @ X_poly) @ X_poly.T @ y
    
    # Gradient Descent
    cache = np.zeros_like(w)
    
    '''m = len(y)  # Number of data points
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_hat = np.dot(X_poly, w) # 1 * X.size()
        error = y_hat - y
        # TODO: Compute gradient
        gradient = (1 / m) * np.dot(X_poly.T, error) # 1 * 3
        # TODO: Update the weights
        w = w - learning_rate * gradient
        
        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 10000 == 0:
            cost = (1 / (2 * m)) * np.sum(np.square(error))
            print(f"Iteration {iteration}, Cost: {cost}")
        
        # SGD
        shuffled_indices = np.random.permutation(len(X_poly))
        X_shuffled = X_poly[shuffled_indices]
        y_shuffled = y[shuffled_indices]
    
        for i in range(0, len(X_poly), batch_size):
            # Mini-batch data
            xi = X_shuffled[i:i + batch_size]
            yi = y_shuffled[i:i + batch_size]
            
            # Compute gradients for the mini-batch
            gradients = 2 / batch_size * xi.T @ (xi @ w - yi)
            
            # Update cache for RMSprop
            cache = decay_rate * cache + (1 - decay_rate) * gradients**2
            
            # Update parameters using RMSprop
            w = w - learning_rate * gradients / (np.sqrt(cache) + epsilon)'''
    
    # store w vector
    with open(coefficient_adv_root, 'w', newline='') as file:
        writer = csv.writer(file)
        for coeff in w:
            writer.writerow([coeff])

    return w

## Make Prediction

In [62]:

def MakePrediction_adv(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO
    # Add polynomial features to X
    X_poly = np.ones((test_dataset.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, test_dataset ** d))  # Add x^d terms to feature matrix
    
    prediction = np.dot(X_poly, w)

    return prediction


In [63]:
# TODO

# (1) Split data
trainData, validationData = SplitData(training_datalist, 0.8)

# (2) Preprocess data
preprocessData = PreprocessData(trainData)
validationData = PreprocessData(validationData)

# (3) Train regression model
coeff = Regression_adv(preprocessData)
training = MakePrediction_adv(coeff, preprocessData[:, used_var])
mape_val = cal_MAPE(preprocessData[:, -1], training)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
validation = MakePrediction_adv(coeff, validationData[:, used_var])
mape_val_1 = cal_MAPE(validationData[:, -1], validation)

# (5) Make prediction of testing dataset and store the values in output_datalist
testing_datalist = np.where(testing_datalist == 'F', 1, np.where(testing_datalist == 'M', 2, testing_datalist))
testing_datalist = testing_datalist.astype(float)
output_datalist = MakePrediction_adv(coeff, testing_datalist[:, used_var])

# Save the prediction result
with open('predictions.csv', 'a', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([degree, mape_val, output_dataroot])

[-29.88171823  -0.05466904   1.34045297   0.37001345   0.03839035
  -0.19844899   0.03351346   0.03619925]
Mape value: 13.05%
Mape value: 16.85%


In [64]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])

# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)